Organizing and Labeling
- Splitting each class's images into 80% train and 90% validation
- Flattening images into train/images and validation/images
- Creating YOLO labels (YOLO txt files) for the images in both training and validation sets
- Generating YAML file 

In [ ]:
import os
import shutil
import random 
from glob import glob
from tqdm import tqdm
from ultralytics import YOLO
import torch
import cv2
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:

# Specifying locations for data:

doodle_data = r"C:\MSAAI\AAI-590\Capstone-Local\Data"
dataset_directory = r"C:\MSAAI\AAI-590\Capstone-Local"
train_image_directory = os.path.join(dataset_directory, "train", "images")
train_label_directory = os.path.join(dataset_directory, "train", "labels")
validation_image_directory = os.path.join(dataset_directory, "validation", "images")
validation_label_directory = os.path.join(dataset_directory, "validation", "labels")

# creating directories for the train and validation sets and their labels:

os.makedirs(train_image_directory, exist_ok=True)
os.makedirs(train_label_directory, exist_ok=True)
os.makedirs(validation_image_directory, exist_ok=True)
os.makedirs(validation_label_directory, exist_ok=True)

In [ ]:
# defining classes

classes = [
    "campfire",
    "cloud",
    "firetruck",
    "helicopter",
    "hospital",
    "mountain",
    "skull",
    "skyscraper",
    "tractor",
    "traffic light",
    "tree",
    "van"
]

In [ ]:
#

for class_id, class_name in enumerate(classes):
    class_folder = os.path.join(doodle_data, class_name)
    if not os.path.isdir(class_folder):
        print(f"Warning: Folder not found for class '{class_name}'")
        continue

    images = glob(os.path.join(class_folder, "*.png"))
    print (f"Found {len(images)} images for class '{class_name}'")

    random.shuffle(images)

    split_index = int(0.8 * len(images))
    train_images = images[:split_index]
    validation_images = images[split_index:]

    for image_path in train_images:
        filename = os.path.basename(image_path)
        base, _ = os.path.splitext(filename)

        dest_img = os.path.join(train_image_directory, filename)
        shutil.copy2(image_path, dest_img)

        label_path = os.path.join(train_label_directory, f"{base}.txt")
        with open(label_path, "w") as f:
            f.write(f"{class_id} 0.5 0.5 1 1\n")

    for image_path in validation_images:
        filename = os.path.basename(image_path)
        base, _ = os.path.splitext(filename)

        # Copy image to val/images
        dest_img = os.path.join(validation_image_directory, filename)
        shutil.copy2(image_path, dest_img)

        # Create YOLO label
        label_path = os.path.join(validation_label_directory, f"{base}.txt")
        with open(label_path, "w") as f:
            f.write(f"{class_id} 0.5 0.5 1 1\n")

print("Dataset split & YOLO labeling complete!")



In [ ]:
# Creating YAML file:

yaml_path = os.path.join(dataset_directory, "data.yaml")
with open(yaml_path, "w") as f:
    f.write(
        f"train: {os.path.join(dataset_directory, 'train', 'images')}\n"
        f"val: {os.path.join(dataset_directory, 'validation', 'images')}\n\n"
        f"nc: {len(classes)}\n"
        f"names: {classes}\n"
    )

print(f"data.yaml created at: {yaml_path}")

Check if your labels are formatted correctly for YOLO

In [ ]:
def check_and_normalize_yolo_labels(label_dir, image_width=640, image_height=640):
    print(f"\nChecking and normalizing: {label_dir}")
    
    txt_files = [f for f in os.listdir(label_dir) if f.endswith(".txt")]
    
    for file in tqdm(txt_files, desc=f"Processing {os.path.basename(label_dir)}", unit="file"):
        path = os.path.join(label_dir, file)
        with open(path, "r") as f:
            lines = f.readlines()

        fixed_lines = []
        needs_fix = False

        for line in lines:
            parts = line.strip().split()
            if len(parts) != 5:
                print(f"Invalid line in {file}: {line.strip()}")
                continue

            try:
                cls = int(parts[0])
                coords = list(map(float, parts[1:]))
            except ValueError:
                print(f"Non-numeric values in {file}: {line.strip()}")
                continue

            # Check if normalization is needed
            if any(val > 1.0 for val in coords):
                needs_fix = True
                x, y, w, h = coords
                x /= image_width
                y /= image_height
                w /= image_width
                h /= image_height
                coords = [x, y, w, h]

            fixed_line = f"{cls} {' '.join(f'{v:.6f}' for v in coords)}"
            fixed_lines.append(fixed_line)

        if needs_fix:
            with open(path, "w") as f:
                f.write("\n".join(fixed_lines) + "\n")

    print(f"Completed: {label_dir}\n")


In [ ]:
check_and_normalize_yolo_labels("yolo_dataset/labels/test")

In [ ]:
check_and_normalize_yolo_labels("yolo_dataset/labels/validation")

In [ ]:
check_and_normalize_yolo_labels("yolo_dataset/labels/train")

Training the YOLO Model 

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Using: {device}')

## 📊 Experiment Log

| Run | Model     | Epochs | Batch | Image Size | Notes                         | mAP@50  | mAP@50-95 |
|-----|-----------|--------|-------|------------|-------------------------------|---------|-----------|
| 1   | yolov8n   | 25     | 16    | 640        | Baseline run with fixed labels| 0.00145 | 0.00039   |
| 2   | yolov8n   | 50     | 16    | 640        | More epochs                   | 0. | 0.   |


In [ ]:
# model training configs

model_size = 'yolov8n.pt'   # or yolov8s.pt for a slightly larger model
epochs = 25                 # increase for better results
batch_size = 32             # increase to stabilize gradients if your GPU can handle it
imgsz = 640                 # match your image size if you prefer (or 128 for speed)
cache = True

# increase for faster data loading speed
# make sure your system has enough ram to use 2 workers (32GB)
# otherwise the training will crash
num_workers = 2

In [ ]:
model = YOLO(f"models/{model_size}")

model.train(
    project="dominic_yolo_runs",
    name="finetuned_model",
    data="yolo_dataset/data.yaml",
    device=device,
    epochs=epochs,
    imgsz=imgsz,
    batch=batch_size,
    lr0=0.01,                   # base learning rate
    optimizer='SGD',            # or 'Adam'
    exist_ok=True,
    workers=num_workers,        # parallel data loading
    cache=cache,                # speeds up subsequent epochs
    plots=True,                 # False --> skips image generation, saving time on training
    verbose=False               # turn off logging
)

Testing on

In [ ]:
# Load your trained YOLO model using the best checkpoint
# model = YOLO("c:/Users/gabri/runs/detect/train17/weights/best.pt")
model = YOLO("dominic_yolo_runs/finetuned_model/weights/best.pt")

# Path to your synthetic test image (640x640 pixels with multiple doodles)
test_image_path = r"image_0016.png"

In [ ]:
# Run inference on the test image
results = model.predict(source=test_image_path, imgsz=640, conf=0.25)

# Get the image with the bounding boxes drawn
image_with_boxes = results[0].plot()

# Display the image using matplotlib
plt.figure(figsize=(8, 8))
plt.imshow(cv2.cvtColor(image_with_boxes, cv2.COLOR_BGR2RGB))
plt.title("Synthetic Data Inference Result")
plt.axis("off")
plt.show()

In [ ]:
model = YOLO("dominic_yolo_runs/finetuned_model/weights/best.pt")

test_image_path = "image_0016.png"  # Adjust path if needed

results = model.predict(source=test_image_path, imgsz=640, conf=0.25)

if len(results) == 0:
    print("No results returned by the model.")
else:
    print(f"Predictions found: {results[0].names}")

    # draw boxes
    output_img = results[0].plot()

    if output_img is None or output_img.size == 0:
        print("Output image is empty.")
    else:
        # save output image
        output_path = 'outputs/images/yolov8_inference_output.png'
        cv2.imwrite(output_path, output_img)
        print(f'Output image saved to: {output_path}')

Automated Tuning


Currently taking a long time for automated turning, so manual tuning is required! See code above with markdown cell to keep track of hyperparameter changes.

In [ ]:
# automated hyperparameter turning
results = model.tune(
    data='yolo_dataset/data.yaml',
    epochs=50,
    iterations=25,
    batch=32,
    imgsz=416,
    plots=False,
    save=True,
    patience=10         # early stopping
)

In [ ]:
tune_results = results.tune_results
df = pd.DataFrame(tune_results)
df = df.sort_values(by='metrics/mAP50(B)', ascending=False)

from datetime import datetime
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
csv_path = f'tuning_results_{timestamp}.csv'
df.to_csv(csv_path, index=False)

print('Tuning results saved to {csv_path}')
df.head()

In [ ]:
best_run_dir = results.best_result.get('save_dir', None)

if best_run_dir and os.path.exists(best_run_dir):
    zip_name = f"best_yolo_run_{timestamp}"
    zip_path = shutil.make_archive(zip_name, 'zip', best_run_dir)
    print(f"Best model zipped to: {zip_path}")
else:
    print("Best run directory not found. Skipping zip.")